<a href="https://colab.research.google.com/github/tahnoktibet/raffle-code/blob/main/Winner_Selection(Royal).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####  Install Libraries ####


!pip install fuzzy_pandas
!pip install --upgrade gspread

In [ ]:
####  Import Libraries  ####


from google.colab import auth
from oauth2client.client import GoogleCredentials

import gspread
import gspread_dataframe as gd
import pandas as pd
import fuzzy_pandas as fpd
import matplotlib.pyplot as plt

In [ ]:
####  Authentication and Spreadsheet Selection  #####

auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())
ss = gc.open_by_key('removedforsafety') #enters spreadsheet key to use for data

In [ ]:
###   Code block 5 - run me to select winners for a given size
###   This block selects the winners, giving preference to their TRSFChance


############################

wsFuzzyCheck = ss.worksheet('FuzzyCheck') #defines FuzzyCheck as correct spreadsheet to export to
wsWinners = ss.worksheet('Winners')       #sets wsWinners to the winners sheet

###   Resets FuzzyCheck & winners sheet to blank
wsFuzzyCheck.clear()
wsWinners.clear()

###   Resets the dfEntry dataframe to the original entry data
dfEntry = pd.DataFrame.from_records(rows[1:],columns=rows[0])

###########################

                    ###THIS SECTION IS IMPORTANT####
                    ###PLEASE ENSURE YOU CHOOSE SIZE
                    #AND SELECT HOW MANY WINNERS
                    #ALONG WITH THE SELECTION METHOD

ordNumberOfWinners = 15   #Number of winners to select (cannot exceed number of entries in size)
ordSize = 'UK9'           #Size to choose - variable must be as listed in MasterSize column (n.b. this is case sensitive and you have to use the following choices for UK6 'UK6: EU39' or 'UK6: EU40')
ordWeight = 'Basechance'  #either 'TRSFChance' or 'Basechance' - CASE SENSITIVE

###########################


###   This section randomly selects the winners based on the weighting

#dfEntry = dfEntry[dfEntry.MasterSize == ordSize]   #old code to filter to single selected size                               #filters sheet to only selected size
#dfEntry = dfEntry.sample(n=ordNumberOfWinners, weights=ordWeight)                #selects from these columns


listWeight = dfEntry['Basechance'].tolist()
dfEntry = dfEntry[ ['Handle' , 'MasterSize' , 'DoubleSub','Comment-Content', 'TRSFMember', 'FollowerStat' , 'TRSFChance','Basechance']] #filters to just show these columns

dfEntry = dfEntry.groupby('MasterSize').sample(n=ordNumberOfWinners,replace=True, weights=listWeight) #runs the selection



wsWinners.update([dfEntry.columns.values.tolist()] + dfEntry.values.tolist())     #updates winners sheet with the winners

###########################

####    This section creates the fuzzy check for the winners list, and exports it straight to the Fuzzy Check Spreadsheet#####
results = fpd.fuzzy_merge(dfEntry, dfEntryBak,
                          left_on=['Comment-Content'],
                          right_on=['Comment-Content'],
                          keep_left=['Handle','FollowerStat','Comment-Content','MasterSize'],
                          keep_right=['Handle','FollowerStat','Comment-Content','MasterSize'],
                          ignore_case=True,
                          method='levenshtein',
                          threshold=0.42)

print("Found",results.shape)
results.head(100)

wsFuzzyCheck.update([results.columns.values.tolist()] + results.values.tolist())
###########################

In [ ]:
# ###   Import the Allocation as a dataframe
# ###   Not needed  yet
# ###   Will be needed in future to automate allocation per sizes

# wsAllo = ss.worksheet('Allo')
# rows = wsAllo.get_all_values()
# dfAllo = pd.DataFrame.from_records(rows[1:],columns=rows[0])

# #####Import the Previous Winners as a dataframe######

# wsPrevWin = ss.worksheet ('prev.win')
# rows = wsEntry.get_all_values()
# dfEntry = pd.DataFrame.from_records(rows[1:],columns=rows[0])


In [ ]:
#results = fpd.fuzzy_merge(dfEntry, dfEntryBak,
#                          left_on=['DoubleSub'],
#                          right_on=['DoubleSub'],
#                          keep_left=['Handle','FollowerStat','DoubleSub','MasterSize'],
#                          keep_right=['Handle','DoubleSub','MasterSize'],
#                          ignore_case=True,
#                          method='levenshtein',
#                          threshold=0.35)
#
#print("Found",results.shape)
#results.head(100)

#wsFuzzyCheck.update([results.columns.values.tolist()] + results.values.tolist())

In [ ]:
#This block adds the winners to the winners sheet - been incorporated in to main code block, please ignore

#wsWinners = ss.worksheet('Winners')
#wsWinners.update([dfEntry.columns.values.tolist()] + dfEntry.values.tolist())




In [ ]:
#####ADDED TO SPREADSHEET BLOCK - CAN IGNORE#######

#####This block provides a Fuzzy Match Output for the winners list compared to all entries####
#####Adjust the threshold to increase/decrease the sensitivity########

#wsFuzzyCheck = ss.worksheet('FuzzyCheck')

# results = fpd.fuzzy_merge(dfEntry, dfEntryBak,
#                           left_on=['DoubleSub'],
#                           right_on=['DoubleSub'],
#                           keep_left=['Handle','DoubleSub','MasterSize'],
#                           keep_right=['Handle','DoubleSub','MasterSize'],
#                           ignore_case=True,
#                           method='levenshtein',
#                           threshold=0.35)

# print("Found",results.shape)
# results.head(100)

#wsFuzzyCheck.update([results.columns.values.tolist()] + results.values.tolist())
